In [5]:
!pip install simplemma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 10.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

tqdm.pandas()

In [104]:
path = '/content/drive/MyDrive/Магистратура/Экспертные и рекомендательные системы/Курсовая работа v2/data/'

items_df = pd.read_csv(path+'items.csv')
items_df.shape

(59599, 5)

In [105]:
items_df.isna().sum()

id             0
title          0
genres        31
authors     6885
year       12879
dtype: int64

In [106]:
items_df[items_df['id'] == 236310]

,id,title,genres,authors,year
22175,236310,Собака Баскервилей. Острие булавки (сборник),"Зарубежные детективы,Классические детективы","Гилберт Кит Честертон,Артур Конан Дойл","1902, 1930, 1932, 1933, 1934,"


# Обработка

## Year

In [107]:
items_df['year'] = items_df['year'].fillna('year_unknown').apply(lambda x: x.split(',')[-1] if x[-1] != ',' else x.split(',')[-2])

## Приведение к леммам

In [108]:
import re
from nltk.corpus import stopwords
import nltk
from simplemma import text_lemmatizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
stop_words_ru = set(stopwords.words('russian'))
stop_words_en = set(stopwords.words('english'))

In [110]:
stop_words = stop_words_ru.union(stop_words_en)

In [111]:
def clean_text(text, stop_words=stop_words, to_lemmatize=True, to_title=False):

  text = re.sub(r'[^\w\s,.]', ' ', text).lower()

  if to_lemmatize:
    text = text_lemmatizer(text, lang=('ru', 'en'))

  else:
    text = text.split(' ')

  # text = [word for word in text if word not in stop_words]

  text = ' '.join(text)
  if to_title:
    text = text.title()

  return text

## authors

In [112]:
items_df['authors'].fillna('authors_unknown', inplace=True)

In [113]:
items_df['authors'] = items_df['authors'].progress_apply(clean_text, to_lemmatize=False, to_title=True)

  0%|          | 0/59599 [00:00<?, ?it/s]

## genres

In [114]:
authors_with_nan_genres = items_df[items_df['genres'].isna()]['authors'].values

In [115]:
genres_author = {}

for author in authors_with_nan_genres:
  genres_author_temp = items_df[items_df['authors'] == author]['genres'].dropna().values
  if len(genres_author_temp) == 0:
    to_replace = 'genres_unknown'
  else:
    author_dct = {}
    for genres in genres_author_temp:
      for genre in genres.split(','):
        author_dct[genre] = author_dct.get(genre, 0) + 1

    to_replace = sorted(author_dct.items(), key=lambda x: x[1], reverse=True)[0][0]
  genres_author[author] = to_replace

In [116]:
def fill_genre(author, genres_author=genres_author):
  to_replace = genres_author[author]
  return to_replace

In [117]:
nan_genre_inds = items_df[items_df['genres'].isna()].index

items_df.loc[nan_genre_inds, 'genres'] = items_df.loc[nan_genre_inds, 'authors'].apply(fill_genre)

In [118]:
for i in tqdm(items_df.index):
  genres = items_df.loc[i, 'genres']
  temp_ = []
  for genre in genres.split(','):
    genre = genre.strip().lower()

    if 'гвэ)' in genre or 'егэ' in genre or 'гиа' in genre:
      genre = 'подготока к экзаменам'

    if ' класс' in genre and 'классика' not in genre:
      genre = genre.split()[:-2]
      genre = ' '.join(genre)

    temp_.append(genre.capitalize())

  temp_ = ','.join(temp_)
  items_df.loc[i, 'genres'] = temp_

  0%|          | 0/59599 [00:00<?, ?it/s]

In [119]:
# items_df['genres'] = items_df['genres'].progress_apply(clean_text, to_lemmatize=True, to_title=False)

## title

In [60]:
# items_df['title'] = items_df['title'].progress_apply(clean_text, to_lemmatize=True, to_title=False)

  0%|          | 0/59599 [00:00<?, ?it/s]

In [121]:
items_df.to_csv(path+'items_clean.csv', index=False)

In [120]:
items_df[items_df['id'] == 236310]

,id,title,genres,authors,year
22175,236310,Собака Баскервилей. Острие булавки (сборник),"Зарубежные детективы,Классические детективы","Гилберт Кит Честертон,Артур Конан Дойл",1934
